In [1]:
from river import datasets
from river import metrics
from river import linear_model
from river import compose
from river import preprocessing
import pickle
import pandas as pd
from river import stream

In [2]:
model = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LinearRegression()
)

In [3]:
df = pd.read_csv('training.csv')
df.head()
df.drop('Unnamed: 0', axis=1, inplace=True)
df['actual_arrival_time'] %= 24*60 

In [4]:
df.tail()

,age,gender,actual_arrival_time,service_time,position
995,27.756518,1,1060.597490,15.226955,1
996,21.772909,2,1074.506654,13.431873,1
997,17.634505,1,1086.760456,12.190352,1
998,42.646958,2,1109.019239,19.694087,0
999,30.525910,2,1126.429603,16.057773,1


In [5]:
def get_samples(df):
    for i in range(len(df)):
        x = {
            'age': int(df.iloc[i]['age']),
            'gender': int(df.iloc[i]['gender']),
            'actual_arrival_time': float(df.iloc[i]['actual_arrival_time']),
            'position': int(df.iloc[i]['position']),
        }
        y = df.iloc[i]['service_time']
        yield x, y

In [10]:
metric = metrics.Accuracy()

for x, y in get_samples(df.iloc[:900, :]):
    y_pred = model.predict_one(x)
    model.learn_one(x, y)
    metric.update(y, y_pred)

In [22]:
with open('model.sav', 'wb') as fh:
        pickle.dump(model, fh)

In [23]:
pickle_off = open ("model.sav", "rb")
model = pickle.load(pickle_off)

In [24]:
x = {'age': 34, 'actual_arrival_time': 0, 'position': 2, 'gender': 1}

In [25]:
model.predict_one(x)

17.268544729462796

In [7]:
# Accuracy Determination
from sklearn.metrics import mean_squared_error

In [11]:
y_true, y_pred = [], []
for x, y in get_samples(df.iloc[:-100, :]):
    y_pred_ = model.predict_one(x)
    y_true.append(y)
    y_pred.append(y_pred_)
    
mean_squared_error(y_true, y_pred, squared=False)

0.08741435033197024